In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
import numpy as np
import os
from glob import glob

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


np.random.seed(3)
tf.random.set_seed(3)

caltech_dir = "./train"
label_name_list = []
data_list = glob('train/*/*.jpg')

def get_label_from_path(path):
    return path.split('\\')[-2]

for path in data_list:
    label_name_list.append(get_label_from_path(path))
categories = np.unique(label_name_list)
nb_classes = len(categories)

image_w = 224
image_h = 224

all_image_paths = []
all_onehot_labels = []

for idx, v in enumerate(categories):
    # 레이블 지정
    label = [0 for i in range(nb_classes)]  # one-hot준비 [0,0,0,0,0]
    label[idx] = 1   # one-hot 리스트 생성
    # 각 폴더에 있는 모든 파일이름에 대한 리스트 생성
    files = glob(caltech_dir + "/" + categories[idx] + "/*.jpg")
    for i,f in enumerate(files):
        all_image_paths.append(f)
        all_onehot_labels.append(label)
        
        if i%20 == 0:
             print(v, " : ", f)
        
def load_image_path_label(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [image_w,image_h])
    image /= 255.0  # normalize to [0,1] range
    return image, label

full_dataset = tf.data.Dataset.from_tensor_slices((all_image_paths, all_onehot_labels))
full_dataset = full_dataset.map(load_image_path_label)        

DATASET_SIZE = len(all_image_paths)

train_size = int(0.75 * DATASET_SIZE)
test_size = DATASET_SIZE - train_size
# 랜덤하게 shuffling
full_dataset = full_dataset.shuffle(buffer_size = int(DATASET_SIZE*1.5))

# 학습 데이터 생성
train_ds = full_dataset.take(train_size)
train_ds = train_ds.batch(16)

# 나머지를 테스트 용으로 사용
test_ds = full_dataset.skip(train_size)
test_ds = test_ds.batch(16)

model = Sequential()

model.add(Conv2D(filters = 64, kernel_size=(3, 3), input_shape=(image_w, image_h, 3), padding="same", activation='relu'))
# model.add(Conv2D(filters = 64, kernel_size=(3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2)) # default
model.add(Dropout(0.3))

model.add(Conv2D(filters = 128, kernel_size=(3, 3), padding="same", activation='relu'))
# model.add(Conv2D(filters = 128, kernel_size=(3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Dropout(0.3))

model.add(Conv2D(filters = 256, kernel_size=(3, 3), padding="same", activation='relu'))
model.add(Conv2D(filters = 256, kernel_size=(3, 3), padding="same", activation='relu'))
# model.add(Conv2D(filters = 256, kernel_size=(3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Dropout(0.3))

model.add(Conv2D(filters = 512, kernel_size=(3, 3), padding="same", activation='relu'))
model.add(Conv2D(filters = 512, kernel_size=(3, 3), padding="same", activation='relu'))
# model.add(Conv2D(filters = 512, kernel_size=(3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Dropout(0.3))

model.add(Conv2D(filters = 512, kernel_size=(3, 3), padding="same", activation='relu'))
model.add(Conv2D(filters = 512, kernel_size=(3, 3), padding="same", activation='relu'))
# model.add(Conv2D(filters = 512, kernel_size=(3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(nb_classes, activation='softmax'))
'''        
model.compile(optimizer = "NAdam",
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
'''
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', 
             metrics = ['accuracy'])

'''
model_dir = './model'

if not os.path.exists(model_dir):
        os.mkdir(model_dir)
        
model_path = model_dir + '{epoch}.model'
checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=6)
'''

model.summary()

10_creova  :  ./train/10_creova\KakaoTalk_20210911_164006583_01.jpg
10_creova  :  ./train/10_creova\KakaoTalk_20210911_164006583_21.jpg
1_ADVIL  :  ./train/1_ADVIL\a2.jpg
1_ADVIL  :  ./train/1_ADVIL\dp_0_0_259.jpg
1_ADVIL  :  ./train/1_ADVIL\dp_0_0_5051.jpg
1_ADVIL  :  ./train/1_ADVIL\dp_0_0_6954.jpg
1_ADVIL  :  ./train/1_ADVIL\dp_0_0_9422.jpg
2_DP  :  ./train/2_DP\dp.jpg
2_DP  :  ./train/2_DP\dp_0_0_294.jpg
2_DP  :  ./train/2_DP\dp_0_0_5574.jpg
2_DP  :  ./train/2_DP\dp_0_0_7179.jpg
2_DP  :  ./train/2_DP\dp_0_0_9696.jpg
3_SUSPEN  :  ./train/3_SUSPEN\s10.jpg
3_SUSPEN  :  ./train/3_SUSPEN\s29.jpg
3_SUSPEN  :  ./train/3_SUSPEN\s47.jpg
3_SUSPEN  :  ./train/3_SUSPEN\s65.jpg
4_beaze  :  ./train/4_beaze\KakaoTalk_20210911_162408561 (1).jpg
4_beaze  :  ./train/4_beaze\KakaoTalk_20210911_162408561_10.jpg
4_beaze  :  ./train/4_beaze\KakaoTalk_20210911_162408561_22.jpg
5_imfectamin  :  ./train/5_imfectamin\KakaoTalk_20210828_215606307.jpg
5_imfectamin  :  ./train/5_imfectamin\KakaoTalk_20210828_2

In [3]:
#model.evaluate(test_ds)
model.fit(train_ds, epochs=50)

model.evaluate(test_ds)

from tensorflow.keras.models import load_model
model.save('test10_img_classification.model')

#history = model.fit(train_ds, epochs=20, callbacks=[checkpoint, early_stopping])
#model.fit(train_ds, epochs = 15)

Epoch 1/50
23/23 [==============================] - 73s 3s/step - loss: 4.2397 - accuracy: 0.1811
Epoch 2/50
23/23 [==============================] - 77s 3s/step - loss: 1.8028 - accuracy: 0.3287
Epoch 3/50
23/23 [==============================] - 80s 3s/step - loss: 1.6346 - accuracy: 0.3454
Epoch 4/50
23/23 [==============================] - 75s 3s/step - loss: 1.4902 - accuracy: 0.3426
Epoch 5/50
23/23 [==============================] - 77s 3s/step - loss: 1.1451 - accuracy: 0.5905
Epoch 6/50
23/23 [==============================] - 75s 3s/step - loss: 1.6297 - accuracy: 0.4791
Epoch 7/50
23/23 [==============================] - 78s 3s/step - loss: 0.9141 - accuracy: 0.6351
Epoch 8/50
23/23 [==============================] - 76s 3s/step - loss: 0.7386 - accuracy: 0.7019
Epoch 9/50
23/23 [==============================] - 75s 3s/step - loss: 1.2818 - accuracy: 0.5460
Epoch 10/50
23/23 [==============================] - 76s 3s/step - loss: 0.6830 - accuracy: 0.6992
Epoch 11/50
23/23 [

In [15]:
from PIL import Image
import os, glob, numpy as np
from tensorflow.keras.models import load_model

import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('./model/test10_img_classification.model') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('./model/model_9.tflite', 'wb') as f:
    f.write(tflite_model)
    
test_dir = './test'
image_w = 224
image_h = 224

X= []
filenames = []
files = glob.glob(test_dir+'/*.jpg')

for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    filenames.append(f)
    X.append(data)

X = np.array(X)
model = load_model('./model/test10_img_classification.model')

prediction = model.predict(X)
np.set_printoptions(formatter={'float':lambda x: "{0:0.3f}".format(x)})
cnt = 0

result = prediction
result


for i in prediction:
    pre_ans = i.argmax()
    print(i)
    print(pre_ans)
    pre_ans_str = ''
    if pre_ans == 0: pre_ans_str = "ADVIL"
    elif pre_ans == 1: pre_ans_str = "DP"
    elif pre_ans == 2: pre_ans_str = "SUSPEN"
    elif pre_ans == 3: pre_ans_str = "beaze"
    elif pre_ans == 4: pre_ans_str = "IMFECTAMIN"
    elif pre_ans == 5: pre_ans_str = "CENTRUM"
    elif pre_ans == 6: pre_ans_str = "kelolef"
    elif pre_ans == 7: pre_ans_str = "selfelinF"
    elif pre_ans == 8: pre_ans_str = "mekain"
    else: pre_ans_str = "creova"

    if i[0] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[1] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[2] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[3] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[4] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[5] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[6] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[7] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[8] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[9] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
   
    
    cnt += 1


[0.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
1
해당 advil.jpg이미지는 DP로 추정됩니다.
[0.000 0.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
2
해당 beaze.jpg이미지는 SUSPEN로 추정됩니다.
[0.000 0.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
2
해당 centrom.jpg이미지는 SUSPEN로 추정됩니다.
[1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
0
해당 crerova.jpg이미지는 ADVIL로 추정됩니다.
[0.000 0.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
2
해당 dp.jpg이미지는 SUSPEN로 추정됩니다.
[0.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000 0.000 0.000]
4
해당 impactamin.jpg이미지는 IMFECTAMIN로 추정됩니다.
[0.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000 0.000 0.000]
4
해당 kalolef.jpg이미지는 IMFECTAMIN로 추정됩니다.
[0.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
1
해당 mekain.jpg이미지는 DP로 추정됩니다.
[0.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
1
해당 selferinF.jpg이미지는 DP로 추정됩니다.
[0.000 0.000 0.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000]
3
해당 suspen.jpg이미지는 beaze로 추정됩니다.
